In [2]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)
from combiners import EpochsCombiner
from typing import *
import mne
import tensorflow as tf
import mneflow as mf
import matplotlib.pyplot as plt
import numpy as np
from utils.data_management import dict2str
from lfcnn_vis import plot_patterns
from LFCNN_decoder import SpatialParameters, TemporalParameters, ComponentsOrder, Predictions, WaveForms
import pickle
from utils.machine_learning import one_hot_decoder
import sklearn.metrics as sm
from utils.machine_learning.confusion import ConfusionEstimator
import scipy as sp
import pylustrator
from PIL import Image

def read_pkl(path: str) -> Any:
    with open(
            path,
            'rb'
        ) as file:
        content = pickle.load(
            file
        )
    return content

2022-06-22 23:42:53.649958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-22 23:42:53.650011: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def draw_sp(
    iy,
    waveforms,
    temporal_parameters,
    spatial_parameters,
    info,
    *,
    logscale=False,
    suptitle=None,
    vmin=None,
    vmax=None,
    flim = 70,
    crop = .05,
    shift = True
):
    data = spatial_parameters.patterns.copy()
    induced = waveforms.induced.copy()[
        iy,
        :flim,
        :
    ]
    crop *= induced.shape[1] / 2

    for i, ind_course in enumerate(induced):
        induced[i] /= ind_course.mean()

    # fig2, axs = plt.subplots(3)
    # ax21, ax22, ax23 = axs
    fig2 = plt.figure(constrained_layout=False)
    gs2 = fig2.add_gridspec(
        nrows=9,
        ncols=2,#3,#2,
        bottom=.1,
        wspace=.05,
        hspace=.1
    )
    ax21 = fig2.add_subplot(gs2[:3, :])
    ax22 = fig2.add_subplot(gs2[3:6, :])#1:2])# :])
    ax23 = fig2.add_subplot(gs2[6:, :])#1:2])# :])
    plot_patterns(data, info, iy, ax21, name_format='', title='')
    ax22_t = ax22.twinx()
    ax22_t.plot(sp.stats.zscore(waveforms.evoked[iy]), '#454545')
    pos = ax22.imshow(np.flip(induced, axis=0), cmap='RdBu_r', vmin=vmin, vmax=vmax)
    cb = fig2.colorbar(pos, ax=ax22, pad=0.2, orientation='horizontal', aspect=75, fraction=.12)
    ax22.set_aspect('auto')
    ax22_t.set_aspect('auto')
    # ax22_t.set_ylim(top=1, bottom=-1)
    ax23.plot(
        temporal_parameters.franges,
        sp.stats.zscore(temporal_parameters.finputs[iy]),
        temporal_parameters.franges,
        sp.stats.zscore(temporal_parameters.foutputs[iy]),
        temporal_parameters.franges,
        sp.stats.zscore(temporal_parameters.fresponces[iy]),
    )
    ax22_t.set_ylabel('Amplitude', labelpad=30, rotation=270)
    ax22_t.spines['top'].set_alpha(.2)
    ax22_t.spines['right'].set_alpha(.2)
    ax22_t.spines['left'].set_alpha(.2)
    ax22_t.spines['bottom'].set_alpha(.2)
    ax22_t.tick_params(axis='both', which='both',length=5, color='#00000050')
    ax22.spines['top'].set_alpha(.2)
    ax22.spines['right'].set_alpha(.2)
    ax22.spines['left'].set_alpha(.2)
    ax22.spines['bottom'].set_alpha(.2)
    ax22.tick_params(axis='both', which='both',length=5, color='#00000050')
    cb.outline.set_color('#00000020')
    cb.ax.tick_params(axis='both', which='both',length=5, color='#00000050')
    times = np.unique(np.round(waveforms.times, 1))

    if shift:
        times = np.round(times - times.mean(), 2)

    ranges = np.linspace(0, len(waveforms.times), len(times)).astype(int)
    ax22.set_xticks(ranges)
    ax22.set_xticklabels(times)
    freqs = [0] + [(i+1) for i in range(waveforms.induced.shape[-2]) if (i+1)%10==0]
    ax22.set_yticks(freqs)
    ax22.set_yticklabels(sorted(freqs, reverse=True))
    ax22.set_xlabel('Time (s)')
    ax22.set_ylabel('Frequency (Hz)', labelpad=15)
    ax23.set_ylim(-2, 11)
    ax23.set_yticks([i for i in range(11) if (i+1)%2])
    ax23.legend(['Filter input', 'Filter output', 'Filter responce'], loc='upper right')
    ax23.spines['top'].set_alpha(.2)
    ax23.spines['right'].set_alpha(.2)
    ax23.spines['left'].set_alpha(.2)
    ax23.spines['bottom'].set_alpha(.2)
    ax23.tick_params(axis='both', which='both',length=5, color='#00000050')
    ax23.set_xlabel('Frequency (Hz)')
    ax23.set_ylabel('Amplitude')
    ax22_t.set_xlim([2 * crop, len(waveforms.times) - 2 * crop])
    ax23.set_xlim([0, 70])

    if logscale:
        ax23.set_yscale('log')
    if suptitle is None:
        fig2.suptitle(f'Latent source {iy + 1}')
    else:
        fig2.suptitle(suptitle)
    plt.show()


In [36]:

def compare(x1 :np.array, x2: np.array) -> float:
    return (x1@x2)/(np.linalg.norm(x1)*np.linalg.norm(x2))

# classification_name = 'RM&RI_vs_LM&LI'
classification_name = 'RM_vs_RI_vs_LM_vs_LI'
# classification_name = 'LM_vs_LI'
# classification_name = 'RM_vs_RI'
subjects_dir = '../Source/Subjects/'
for subject in os.listdir(subjects_dir):
    if subject == 'Pse_Udo':
        continue
    subject_info_path = f'../Source/Subjects/{subject}/Info'
    info = read_pkl(os.path.join(subject_info_path, os.listdir(subject_info_path)[0]))
    info.pick_channels(
        list(
            filter(
                lambda ch_name: (ch_name[-1] == '2' or ch_name[-1] == '3') and 'meg' in ch_name.lower(),
                info['ch_names']
            )
        )
    )
    path1 = f'../Source/Subjects/{subject}/LFCNN_train_B1-B3/Parameters'
    path2 = f'../Source/Subjects/{subject}/LFCNN_train_B10-B12/Parameters'
    temporal_parameters1 = read_pkl(os.path.join(path1, f'{classification_name}_temporal.pkl'))
    temporal_parameters2 = read_pkl(os.path.join(path2, f'{classification_name}_temporal.pkl'))
    spatial_parameters1 = read_pkl(os.path.join(path1, f'{classification_name}_spatial.pkl'))
    spatial_parameters2 = read_pkl(os.path.join(path2, f'{classification_name}_spatial.pkl'))
    waveforms1 = read_pkl(os.path.join(path1, f'{classification_name}_waveforms.pkl'))
    waveforms2 = read_pkl(os.path.join(path2, f'{classification_name}_waveforms.pkl'))

    patterns = spatial_parameters1.patterns
    plt.rcParams.update({'font.size': 22})

    indices1 = list(
        map(
            lambda item: item[0],
            sorted(
                enumerate(spatial_parameters1.patterns.T),
                key=lambda item: np.abs(item[1]).sum(0),
                reverse=True
            )
        )
    )

    pair_found = False
    pat_i = 0
    while not pair_found:
        best_pattern_index = indices1[pat_i]
        best_pattern = patterns[:, best_pattern_index]

        indices2 = list(
            map(
                lambda item: item[0],
                sorted(
                    enumerate(spatial_parameters2.patterns.T),
                    key=lambda item: compare(best_pattern, item[1]),
                    reverse=True
                )
            )
        )
        ratio = compare(best_pattern, spatial_parameters2.patterns[:, indices2[0]])
        if ratio >= .75:
            pair_found = True
        else:
            pat_i += 1

    path1 = f'/home/user/Downloads/Pics/Dumb/{subject}pictemp1.jpg'
    path2 = f'/home/user/Downloads/Pics/Dumb/{subject}pictemp2.jpg'
    path3 = f'/home/user/Downloads/Pics/Res/{subject}.jpg'

    induced1 = waveforms1.induced[indices1[pat_i], :, :]
    induced2 = waveforms2.induced[indices1[pat_i], :, :]
    for induced in [induced1, induced2]:
        for i, ind_course in enumerate(induced):
            induced[i] /= ind_course.mean()
    vmax = max(induced1.max(), induced2.max())
    vmin = min(induced1.min(), induced2.min())
    %matplotlib qt
    draw_sp(indices1[pat_i], waveforms1, temporal_parameters1, spatial_parameters1, info, suptitle=f'{subject} B1-B3', vmin=vmin, vmax=vmax)

    w, h = 12, 15

    fig = plt.gcf()
    fig.set_size_inches(w, h)
    plt.savefig(path1)
    plt.close()

    draw_sp(indices2[0], waveforms2, temporal_parameters2, spatial_parameters2, info, suptitle=f'{subject} B10-B12 '
                f'({ratio : .3f})', vmin=vmin, vmax=vmax)

    fig = plt.gcf()
    fig.set_size_inches(w, h)
    plt.savefig(path2)
    plt.close()

    pylustrator.load(path1)
    pylustrator.load(path2, offset=[1, 0])
    plt.savefig(path3)
    plt.close()

images = [
    Image.open('/home/user/Downloads/Pics/Res/' + f)
    for f in os.listdir('/home/user/Downloads/Pics/Res')
]
pdf_path = f'/home/user/Downloads/Pics/{classification_name}_st_output.pdf'
images[0].save(
    pdf_path, "PDF", resolution=100.0, save_all=True, append_images=images[1:]
)


## Log
#### Components indices are written in brackets

* Az_Mar_05, B7-B8, The most frequent components (29, 17) has small sum of weights and noisy patterns, so components with the highest sum pf weights (30, 12) have been chosen.
* Fe_To_08, B7-B8, The most frequent component is (7), but (13, 17) has been chosen, as they have bigger sum of weights.
* Ku_EL_09, B1-B2, One of the most frequent components (27) has very low sum of weights
* Ku_EL_09, B7-B8, One of the most frequent components (19) has very low sum of weights, as well as one less frequent component (15)
* Pr_An_04, B1-B2, The most frequent components (25, 28, 0, 9) have low sum of weights
* Ru_Ek_07, B7-B8, The most frequent component (7) has low sum of weights
* Se_Tu_03, B1-B2, The most frequent component (25) has low sum of weights
* Se_Tu_03, B7-B8, The most frequent component (25) has low sum of weights
* Te_Ali_11, B1-B2, The most frequent components (24, 22) have low sum of weights

In [4]:
%matplotlib qt
plt.rcParams.update({'font.size': 22})
subject_name = 'Fe_To'
path = '/home/user/Downloads/Pics/Dumb/sample1.png'
i=2
from lfcnn_vis import *
from collections import Counter
cases_to_combine = ['LM', 'LI']
option = 1
option = {1: 'B1-B3', 2: 'B10-B12'}[option]

classification_name,\
    classification_postfix,\
    classification_prefix = None, '', ''
cases_to_combine = [case.split(' ') for case in cases_to_combine]

cases_to_combine = sorted(cases_to_combine, reverse=True)
class_names = ['&'.join(sorted(cases_combination, reverse=True)) for cases_combination in cases_to_combine]

if classification_name is None:
    classification_name = '_vs_'.join(class_names)

# model = f'LFCNN_train_{option}'
model='LFCNN'
subjects_dir = os.path.join('../', 'Source', 'Subjects')
classification_name_formatted = "_".join(list(filter(lambda s: s not in (None, ""), [classification_prefix, classification_name, classification_postfix])))
subject_info_path = os.path.join(subjects_dir, subject_name, 'Info')
subject_parameters_path = os.path.join(subjects_dir, subject_name, model, 'Parameters')
spatial_parameters = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_spatial.pkl'))
temporal_parameters = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_temporal.pkl'))
waveforms = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_waveforms.pkl'))
orders = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_sorting.pkl'))
# all_orders = dict(sorted(Counter(np.matrix.flatten(np.squeeze([orders[order_name] for order_name in range(len(orders))]))).items(), key=lambda x: x[1], reverse=True))

info = read_pkl(os.path.join(subject_info_path, os.listdir(subject_info_path)[0]))
info.pick_channels(
    list(
        filter(
            lambda ch_name: (ch_name[-1] == '2' or ch_name[-1] == '3') and 'meg' in ch_name.lower(),
            info['ch_names']
        )
    )
)

draw_sp(i, waveforms, temporal_parameters, spatial_parameters, info, suptitle='')
w = 10
h = 15
fig = plt.gcf()
fig.set_size_inches(w, h)
plt.savefig(path)

(204, 32)

In [14]:
subjects_dir = os.path.join('../', 'Source', 'Subjects')

def compare(x1 :np.array, x2: np.array) -> float:
    return (x1@x2)/(np.linalg.norm(x1)*np.linalg.norm(x2))

for cases_to_combine in [
    ['LM LI', 'RM RI'],
    ['LM', 'LI', 'RM', 'RI'],
    ['LM', 'LI'],
    ['RM', 'RI']
]:
    classification_name,\
        classification_postfix,\
        classification_prefix = None, '', ''
    cases_to_combine = [case.split(' ') for case in cases_to_combine]
    cases_to_combine = sorted(cases_to_combine, reverse=True)
    class_names = ['&'.join(sorted(cases_combination, reverse=True)) for cases_combination in cases_to_combine]

    if classification_name is None:
        classification_name = '_vs_'.join(class_names)

    classification_name_formatted = "_".join(list(filter(lambda s: s not in (None, ""), [classification_prefix, classification_name, classification_postfix])))

    subjects_induceds1, subjects_induceds2, subject_induced_diffs = list(), list(), list()
    for subject_name in os.listdir(subjects_dir):

        if subject_name == 'Pse_Udo':
            continue

        induceds_pair = list()
        ses = 'B1-B3'

        model = f'LFCNN_train_{ses}'

        subject_parameters_path = os.path.join(subjects_dir, subject_name, model, 'Parameters')

        waveforms = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_waveforms.pkl'))
        spatial_parameters = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_spatial.pkl'))
        sums = spatial_parameters.patterns.sum(0)

        induceds_pair.append(list(
            map(
                lambda item: np.array(item[1]),
                sorted(
                    enumerate(
                        waveforms.induced.tolist()
                    ),
                    key=lambda item: sums[item[0]],
                    reverse=True
                )
            )
        ))

        ses = 'B10-B12'

        model = f'LFCNN_train_{ses}'

        subject_parameters_path = os.path.join(subjects_dir, subject_name, model, 'Parameters')

        waveforms = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_waveforms.pkl'))
        spatial_parameters = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_spatial.pkl'))
        sums = spatial_parameters.patterns.sum(0)


        induceds1, induceds2, tfs = list(), list(), list()
        for induced1, induced2 in zip(*induceds_pair):
            for induced in [induced1, induced2]:
                for i, freq in enumerate(induced):
                    induced[i] /= freq.mean()

            tf = (induced1 - induced2)/induced1

            # tf = np.abs(induced1 - induced2)/induced1

            # for induced in [induced1, induced2, tf]:
            #     for i, freq in enumerate(induced):
            #         induced[i] /= freq.mean()

            induceds1.append(induced1)
            induceds2.append(induced2)
            tfs.append(tf)

        subjects_induceds1.append(np.array(induceds1))
        subjects_induceds2.append(np.array(induceds2))
        subject_induced_diffs.append(np.array(tfs))

    induceds1_ave = np.array(subjects_induceds1).mean(0)
    induceds2_ave = np.array(subjects_induceds2).mean(0)
    induced_diffs_ave = np.array(subject_induced_diffs).mean(0)
    path = '/home/user/Downloads/Pics/Dumb'

    for i, (induced1, induced2, tf) in enumerate(zip(induceds1_ave, induceds2_ave, induced_diffs_ave)):

        fig, (ax1, ax2, ax3) = plt.subplots(ncols=3)
        ax1.imshow(
            induced1,
            cmap='RdBu_r',
            origin='lower'
        )
        ax1.set_title('B1-B3')
        ax2.imshow(
            tf,
            cmap='RdBu_r',
            origin='lower'
        )
        ax2.set_title('Diff')
        ax3.imshow(
            induced2,
            cmap='RdBu_r',
            origin='lower'
        )
        ax3.set_title('B10-B12')
        w, h = 15, 5
        fig = plt.gcf()
        fig.set_size_inches(w, h)
        plt.savefig(os.path.join(path, f'pictemp{i}.jpg'), dpi=200)
        plt.close()

    images = [
        Image.open(os.path.join(path, f))
        for f in os.listdir(path)
    ]
    pdf_path = f'/home/user/Downloads/Pics/sep_cl_diff/{classification_name_formatted}_sep_cl_diff.pdf'
    images[0].save(
        pdf_path, "PDF", resolution=100.0, save_all=True, append_images=images[1:]
    )

In [14]:
subjects_dir = os.path.join('../', 'Source', 'Subjects')

def draw_inddif(induced1, induced2, tf, savepath, title1='B1-B3', title2='Diff', title3='B10-B12', suptitle=''):

    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3)
    ax1.imshow(
        induced1,
        cmap='RdBu_r',
        origin='lower'
    )
    ax1.set_title(title1)
    ax2.imshow(
        tf,
        cmap='RdBu_r',
        origin='lower'
    )
    ax2.set_title(title2)
    ax3.imshow(
        induced2,
        cmap='RdBu_r',
        origin='lower'
    )
    ax3.set_title(title3)
    w, h = 15, 5
    fig.suptitle(suptitle)
    fig.set_size_inches(w, h)
    plt.savefig(savepath, dpi=200)
    plt.close()

def compare(x1 :np.array, x2: np.array) -> float:
    return (x1@x2)/(np.linalg.norm(x1)*np.linalg.norm(x2))

for cases_to_combine in [
    ['LM LI', 'RM RI'],
    ['LM', 'LI', 'RM', 'RI'],
    ['LM', 'LI'],
    ['RM', 'RI']
]:
    classification_name,\
        classification_postfix,\
        classification_prefix = None, '', ''
    cases_to_combine = [case.split(' ') for case in cases_to_combine]
    cases_to_combine = sorted(cases_to_combine, reverse=True)
    class_names = ['&'.join(sorted(cases_combination, reverse=True)) for cases_combination in cases_to_combine]

    if classification_name is None:
        classification_name = '_vs_'.join(class_names)

    classification_name_formatted = "_".join(list(filter(lambda s: s not in (None, ""), [classification_prefix, classification_name, classification_postfix])))

    tfs = list()
    induceds1 = list()
    induceds2 = list()
    for subject_name in os.listdir(subjects_dir):

        if subject_name == 'Pse_Udo':
            continue

        ses = 'B1-B3'
        model = f'LFCNN_train_{ses}'
        subject_parameters_path = os.path.join(subjects_dir, subject_name, model, 'Parameters')
        waveforms1 = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_waveforms.pkl'))
        spatial_parameters1 = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_spatial.pkl'))
        ses = 'B10-B12'
        model = f'LFCNN_train_{ses}'
        subject_parameters_path = os.path.join(subjects_dir, subject_name, model, 'Parameters')
        waveforms2 = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_waveforms.pkl'))
        spatial_parameters2 = read_pkl(os.path.join(subject_parameters_path, f'{classification_name_formatted}_spatial.pkl'))
        patterns = spatial_parameters1.patterns

        indices1 = list(
            map(
                lambda item: item[0],
                sorted(
                    enumerate(spatial_parameters1.patterns.T),
                    key=lambda item: np.abs(item[1]).sum(0),
                    reverse=True
                )
            )
        )

        pair_found = False
        pat_i = 0
        while not pair_found:
            best_pattern_index = indices1[pat_i]
            best_pattern = patterns[:, best_pattern_index]

            indices2 = list(
                map(
                    lambda item: item[0],
                    sorted(
                        enumerate(spatial_parameters2.patterns.T),
                        key=lambda item: compare(best_pattern, item[1]),
                        reverse=True
                    )
                )
            )
            ratio = compare(best_pattern, spatial_parameters2.patterns[:, indices2[0]])
            if ratio >= .75:
                pair_found = True
            else:
                pat_i += 1

        induced1 = waveforms1.induced[indices1[pat_i], :, :]
        induced2 = waveforms2.induced[indices1[pat_i], :, :]

        for induced in [induced1, induced2]:
            for i, ind_course in enumerate(induced):
                induced[i] /= ind_course.mean()

        # tf = (induced1 - induced2)/induced1
        tf = np.abs(induced1 - induced2)/induced1

        tfs.append(tf)
        induceds1.append(induced1)
        induceds2.append(induced2)

        draw_inddif(induced1, induced2, tf, f'/home/user/Downloads/Pics/Dumb/{subject_name}.jpg', title3=f'B10-B12, {ratio}', suptitle=subject_name)

    draw_inddif(np.array(induceds1).mean(0), np.array(induceds2).mean(0), np.array(tfs).mean(0), f'/home/user/Downloads/Pics/Dumb/average.jpg', title3=f'B10-B12', suptitle='average')
    images = [
        Image.open(os.path.join('/home/user/Downloads/Pics/Dumb', f))
        for f in os.listdir('/home/user/Downloads/Pics/Dumb')
    ]
    pdf_path = f'/home/user/Downloads/Pics/sep_cl_diff_abs/{classification_name_formatted}_sep_cl_diff.pdf'
    images[0].save(
        pdf_path, "PDF", resolution=100.0, save_all=True, append_images=images[1:]
    )

In [11]:
from utils.storage_management import check_path


In [14]:

def compare(x1 :np.array, x2: np.array) -> float:
    return (x1@x2)/(np.linalg.norm(x1)*np.linalg.norm(x2))
subjects_dir = '../Source/Subjects/'

for classification_name in [
    'RM&RI_vs_LM&LI',
    'RM_vs_RI_vs_LM_vs_LI',
    'LM_vs_LI',
    'RM_vs_RI'
]:

    classification_savepath = os.path.join('/home/user/Downloads/SpatioTemporal', classification_name)
    check_path(classification_savepath)
    for subject in os.listdir(subjects_dir):
        if subject == 'Pse_Udo':
            continue
        subject_savepath = os.path.join(classification_savepath, subject)
        check_path(subject_savepath)
        subject_info_path = f'../Source/Subjects/{subject}/Info'
        info = read_pkl(os.path.join(subject_info_path, os.listdir(subject_info_path)[0]))
        info.pick_channels(
            list(
                filter(
                    lambda ch_name: (ch_name[-1] == '2' or ch_name[-1] == '3') and 'meg' in ch_name.lower(),
                    info['ch_names']
                )
            )
        )
        path = f'../Source/Subjects/{subject}/LFCNN/Parameters'
        temporal_parameters = read_pkl(os.path.join(path, f'{classification_name}_temporal.pkl'))
        spatial_parameters = read_pkl(os.path.join(path, f'{classification_name}_spatial.pkl'))
        waveforms = read_pkl(os.path.join(path, f'{classification_name}_waveforms.pkl'))

        patterns = spatial_parameters.patterns
        plt.rcParams.update({'font.size': 22})

        indices1 = list(
            map(
                lambda item: item[0],
                sorted(
                    enumerate(patterns.T),
                    key=lambda item: np.abs(item[1]).sum(0),
                    reverse=True
                )
            )
        )

        for i in range(5):
            %matplotlib qt
            draw_sp(indices1[i], waveforms, temporal_parameters, spatial_parameters, info, suptitle=f'{subject} B1-B3')

            w, h = 12, 15
            fig = plt.gcf()
            fig.set_size_inches(w, h)
            plt.savefig(os.path.join(subject_savepath, f'pattern{i}.png'), dpi=300)
            plt.close()


